# Introduction to Sagemaker Experiment Logger for PyTorch-Lightning
 This introduction shows the usage of the [SagemakerExperimentsLogger](https://tsenst.github.io/lightning-experiments-logger/api_logger.html#experiments_addon.logger.SagemakerExperimentsLogger) on the example of simple MNIST classification using [Trainer](https://lightning.ai/docs/pytorch/stable/common/trainer.html) class of the PyTorch Lightning API.

## Setup
To run the notebook to following packages have to be installed

In [ ]:
! pip install --quiet "setuptools>=67.4.0" "scikit-learn>=1.3.1" "pytorch-lightning>=2.0.0" "sagemaker>=2.190.0" "torchvision>=0.16.0" "torchmetrics>=0.11.4"

## Prerequisite
At first the dataset has to be loaded. The used model has been introduced by the [Simple MNIST Model](https://lightning.ai/docs/pytorch/stable/notebooks/lightning_examples/mnist-hello-world.html) example of the Pytorch-Lightning page.


In [ ]:
import os

from pytorch_lightning import Trainer
from sagemaker.experiments.run import Run

from example.model import MNISTModel
from experiments_addon.logger import SagemakerExperimentsLogger

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
mnist_model = MNISTModel(data_dir=PATH_DATASETS)

## How to Use
The log with [SagemakerExperimentsLogger](https://github.com/tsenst/lightning-experiments-logger/blob/main/experiments_addon/logger.py#L44) to AWS SageMaker Experiments. Three different use cases will be discussed:
1. Use logger inside the run context
2. Explicitly passing *run_name* and *experiment_name*
3. Use logger with other loggers

Sagemaker Experiments are organized by experiments and run. A run is a subunit of an experiment. When tracking the metrics of a training run the run (given by *run_name*) and experiment (given by *experiment_name*  has to be provided. For the experiment tracking SageMake API defines a Run Context within the metric writing code has been implemented.
 
### 1. A Logger within a Run Context
For the use in notebook it is recommendation to [create a Run object](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments-create.html). All log methods have to be called inside *with* statement. Because the *Trainer* calls the logger log methods internally, it needs to be within the *with* statement too. The *SagemakerExperimentLogger* object will be called without providing *experiment_name* and *run_name* since both properties are retrieved inside the logger from the run context. 

In [ ]:
with Run(experiment_name="testExperiment", run_name="testRun1"):
    logger = SagemakerExperimentsLogger()
    trainer = Trainer(
        logger=logger,
        max_epochs=3,
    )
    trainer.fit(mnist_model)
    trainer.test()

If the logger will be applied inside a [Training Job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-training.html) the logger can be called without any arguments, since the logger used the experiment configuration from the run context of the Training Job.

### 2. A Logger by Explicitly Passing in run_name and experiment_name
There may be cases where the *with Run* statement is not practical. For that reason it is possible to obtain the run context at each log operation. To run the *SagemakerExperimentsLogger* without the *with* statement provide *experiment_name* and *run_name* when creating the object. 

In [ ]:
logger = SagemakerExperimentsLogger(
    experiment_name="TestExp",
    run_name="TestRun"
)

trainer = Trainer(
    logger=logger,
    max_epochs=3,
)
trainer.fit(mnist_model)
trainer.test()

### 3. Multiple Loggers
PyTorch Lightning *Trainer* class allows to apply multiple model. To do so pass a list of logger to the *Trainer* object.

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

tensorboard_logger = TensorBoardLogger()
with Run(experiment_name="testExperiment", run_name="testRun2"):
    logger = SagemakerExperimentsLogger()
    trainer = Trainer(
        logger=[logger, tensorboard_logger],
        max_epochs=3,
    )
    trainer.fit(mnist_model)
    trainer.test()

## Conclusion
Congratulations! You have trained a MNSIT model and logged the evaluation metrics and hyperparameters to SageMaker Experiment API. You should now be able to see the plots in the SageMaker Studio of your AWS account. 